## Instalasi Library

In [1]:
pip install pandas sqlalchemy psycopg2-binary pyod pycaret

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from sqlalchemy import create_engine
import traceback

## # --- Bagian 1: Pengaturan Koneksi Database ---

In [3]:
# --- 1. IMPORT PUSTAKA YANG DIBUTUHKAN ---
import pandas as pd
from sqlalchemy import create_engine  # <-- PENTING: Import untuk fungsi create_engine
import traceback                    # <-- PENTING: Import untuk traceback.print_exc()

# --- (Pastikan variabel koneksi Anda sudah ada di sini) ---
# Contoh:
db_user = 'posdtgres'
db_password = '123456789'
db_host = 'localhost'
db_port = '5432'
db_name = 'iris_data2'
table_name = 'iris_data2'
db_url = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'




In [4]:
import pandas as pd
import io

# --- 1. MEMBUAT DATA FAKE UNTUK DEMO ---
# Kita akan membuat data dummy dalam bentuk string untuk simulasi file CSV
csv_data = """id,nama,usia,kota
1,Andi,25,Jakarta
2,Budi,30,Surabaya
3,Citra,22,Bandung
4,Dewi,28,Yogyakarta
5,Eko,35,Semarang
"""

# Menggunakan io.StringIO untuk membaca string seolah-olah itu adalah file
# Ini menghindari ketergantungan pada file eksternal
csv_file_like = io.StringIO(csv_data)

In [5]:
# deteksi_outlier_abod.py
import os
import numpy as np
import pandas as pd
from pyod.models.abod import ABOD
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# ---------- 1) Persiapan df_original ----------
# Jika kamu sudah punya df_original di memori (mis. dari pd.read_csv),
# script ini otomatis pakai. Kalau tidak, script akan mencoba membaca
# 'data.csv' di folder kerja, jika tidak ada akan membuat contoh data.
try:
    df_original  # check if exists
except NameError:
    df_original = None

if df_original is None:
    csv_path = "data.csv"  # ganti path kalau kamu punya file lain
    if os.path.exists(csv_path):
        df_original = pd.read_csv(csv_path)
        print(f"Loaded dataset from {csv_path} (shape={df_original.shape})")
    else:
        print("df_original tidak ditemukan dan 'data.csv' juga tidak ada. Membuat contoh data untuk demo.")
        rng = np.random.RandomState(42)
        df_original = pd.DataFrame({
            "id": np.arange(100),
            "x1": rng.normal(0, 1, 100),
            "x2": rng.normal(5, 2, 100),
            "category": ["A"]*50 + ["B"]*50
        })
        # tambahkan beberapa outlier buat demo
        df_original.loc[0, "x1"] = 20
        df_original.loc[1, "x2"] = -30

# ---------- 2) Ambil kolom numerik ----------
df_numeric = df_original.select_dtypes(include=[np.number])
if df_numeric.shape[1] == 0:
    raise ValueError("Tidak ada kolom numerik di df_original. ABOD butuh kolom numerik untuk bekerja.")

# Pastikan tidak ada nilai infinite; ganti dengan NaN lalu impute
df_numeric = df_numeric.replace([np.inf, -np.inf], np.nan)

# ---------- 3) Imputasi & scaling (recommended) ----------
imputer = SimpleImputer(strategy="mean")
X_imputed = imputer.fit_transform(df_numeric)  # bentuk: (n_samples, n_features)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# ---------- 4) Inisialisasi dan fit ABOD ----------
# Catatan: ABOD bisa lambat untuk dataset sangat besar (kompleksitas tinggi).
abod_model = ABOD(contamination=0.05)  # ubah contamination sesuai perkiraan outlier
abod_model.fit(X_scaled)

# ---------- 5) Ambil hasil ----------
labels = abod_model.labels_.astype(int)            # 0 = inlier, 1 = outlier
scores = abod_model.decision_scores_               # nilai anomali (lebih besar = lebih anomali)

# Pasang hasil ke DataFrame asli (preserve index)
labels_series = pd.Series(labels, index=df_numeric.index, name="outlier_abod")
scores_series = pd.Series(scores, index=df_numeric.index, name="abod_score")

df_with_outliers = df_original.copy()
df_with_outliers["outlier_abod"] = labels_series
df_with_outliers["abod_score"] = scores_series

# ---------- 6) Tampilkan ringkasan ----------
n_outliers = int(df_with_outliers["outlier_abod"].sum())
print(f"\n--- Hasil Deteksi ABOD ---")
print(f"Jumlah baris total : {len(df_with_outliers)}")
print(f"Jumlah outlier terdeteksi: {n_outliers}\n")

if n_outliers > 0:
    print("Contoh baris yang dideteksi sebagai outlier (maks 10):")
    print(df_with_outliers[df_with_outliers["outlier_abod"] == 1].head(10))
else:
    print("Tidak ada outlier terdeteksi dengan threshold contamination saat ini.")

# ---------- 7) Buat dataset bersih tanpa outlier ----------
df_bersih_abod = df_with_outliers[df_with_outliers["outlier_abod"] == 0].copy()
# hapus kolom bantu bila tidak perlu
df_bersih_abod.drop(columns=["outlier_abod", "abod_score"], inplace=True)

print(f"\nJumlah baris setelah menghapus outlier: {len(df_bersih_abod)}")

# Jika mau, simpan hasil:
# df_with_outliers.to_csv("with_outliers_abod.csv", index=False)
# df_bersih_abod.to_csv("clean_abod.csv", index=False)


df_original tidak ditemukan dan 'data.csv' juga tidak ada. Membuat contoh data untuk demo.

--- Hasil Deteksi ABOD ---
Jumlah baris total : 100
Jumlah outlier terdeteksi: 5

Contoh baris yang dideteksi sebagai outlier (maks 10):
    id         x1         x2 category  outlier_abod    abod_score
0    0  20.000000   2.169259        A             1 -1.823226e-07
1    1  -0.138264 -30.000000        A             1 -9.315983e-08
6    6   1.579213   8.772372        A             1 -9.938134e-02
13  13  -1.913280   9.926484        A             1 -3.822176e-02
79  79  -1.987569  10.440338        B             1 -3.302782e-02

Jumlah baris setelah menghapus outlier: 95


In [6]:
# detect_outliers_abod.py
import traceback
import pandas as pd
import numpy as np

# ----------------- Pilihan: Buat contoh data bila df_original belum tersedia -----------------
try:
    df_original
except NameError:
    df_original = None

if df_original is None:
    print("⚠️ df_original tidak ditemukan. Membuat contoh data untuk demo.")
    rng = np.random.RandomState(42)
    df_original = pd.DataFrame({
        "id": np.arange(100),
        "x1": rng.normal(0, 1, 100),
        "x2": rng.normal(5, 2, 100),
        "kategori": ["A"]*50 + ["B"]*50
    })
    # tambahkan beberapa outlier utk demo
    df_original.loc[0, "x1"] = 20
    df_original.loc[1, "x2"] = -30

# ----------------- Helper: mapping hasil ke df_original -----------------
def map_predictions_to_original(df_orig, processed_df_with_origidx, anomaly_col='Anomaly', score_col='Anomaly_Score'):
    """
    processed_df_with_origidx : DataFrame hasil pycaret/pyod yang memiliki kolom '_orig_index'
    """
    df_out = df_orig.copy()
    # buat kolom target jika belum ada
    df_out[anomaly_col] = np.nan
    df_out[score_col] = np.nan

    # lakukan mapping berdasarkan _orig_index (harus ada di processed_df_with_origidx)
    idxs = processed_df_with_origidx['_orig_index'].values
    df_out.loc[idxs, anomaly_col] = processed_df_with_origidx[anomaly_col].values
    df_out.loc[idxs, score_col] = processed_df_with_origidx[score_col].values
    # ubah type Anomaly jadi int untuk baris yang bukan NaN
    df_out.loc[df_out[anomaly_col].notna(), anomaly_col] = df_out.loc[df_out[anomaly_col].notna(), anomaly_col].astype(int)
    return df_out

# ----------------- 1) Siapkan kolom numerik (tetap simpan index asli) -----------------
df_numeric = df_original.select_dtypes(include=[np.number]).copy()
if df_numeric.shape[1] == 0:
    raise ValueError("❌ Tidak ada kolom numerik di df_original. ABOD butuh kolom numerik.")

# simpan index asli agar bisa di-map kembali
df_numeric['_orig_index'] = df_numeric.index
df_numeric = df_numeric.reset_index(drop=True)  # pycaret/pyod bekerja pada 0..n-1

# Buat versi input untuk model (hapus kolom helper)
model_input = df_numeric.drop(columns=['_orig_index'])

# ----------------- 2) Coba PyCaret (jika tersedia) -----------------
try:
    from pycaret.anomaly import setup, create_model, assign_model
    print("\n=== Mencoba PyCaret ABOD ===")
    # exclude 'id' jika ada, karena id sering tidak berguna untuk deteksi outlier
    ignore_features = []
    if 'id' in model_input.columns:
        ignore_features = ['id']

    # Setup
    exp = setup(
        data=model_input,
        silent=True,
        html=False,      # agar tidak coba render HTML (berguna di script)
        session_id=123,
        ignore_features=ignore_features
    )

    # create_model: untuk PyCaret anomaly parameter proporsi outlier bernama 'fraction'
    abod_py = create_model('abod', fraction=0.05)

    # assign_model: menambahkan kolom 'Anomaly' (1 = outlier) dan 'Anomaly_Score'
    pred = assign_model(abod_py)

    # Pastikan kolom ada
    if 'Anomaly' not in pred.columns or 'Anomaly_Score' not in pred.columns:
        raise RuntimeError("PyCaret tidak mengembalikan kolom 'Anomaly'/'Anomaly_Score' seperti yang diharapkan.")

    # tambahkan _orig_index agar bisa di-map kembali
    pred['_orig_index'] = df_numeric['_orig_index'].values

    # rename/standarisasi kolom agar konsisten
    pred = pred.rename(columns={'Anomaly': 'Anomaly', 'Anomaly_Score': 'Anomaly_Score'})

    # map ke df_original
    df_with_outliers = map_predictions_to_original(df_original, pred, anomaly_col='Anomaly', score_col='Anomaly_Score')

    # ringkasan
    n_out = int(df_with_outliers['Anomaly'].sum())
    print(f"PyCaret ABOD selesai — ditemukan {n_out} outlier dari {len(df_with_outliers)} baris.")
    print("Contoh outlier (max 10):")
    print(df_with_outliers[df_with_outliers['Anomaly'] == 1].head(10))

    # buat dataset bersih
    df_clean = df_with_outliers[df_with_outliers['Anomaly'] != 1].copy()
    # hapus kolom bantu jika mau
    df_clean = df_clean.drop(columns=['Anomaly', 'Anomaly_Score'])
    print(f"Jumlah baris setelah bersih: {len(df_clean)}")

except Exception as e_py:
    print("\n⚠️ PyCaret gagal atau tidak tersedia. Error:")
    traceback.print_exc()
    # ----------------- 3) Fallback: PyOD ABOD -----------------
    try:
        print("\n=== Menggunakan PyOD ABOD (fallback) ===")
        from pyod.models.abod import ABOD
        from sklearn.impute import SimpleImputer
        from sklearn.preprocessing import StandardScaler

        # Imputasi dan scaling
        X = model_input.replace([np.inf, -np.inf], np.nan).values
        imputer = SimpleImputer(strategy='mean')
        X_imp = imputer.fit_transform(X)
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_imp)

        # fit ABOD
        abod = ABOD(contamination=0.05)
        abod.fit(X_scaled)

        labels = abod.labels_.astype(int)            # 0 inlier, 1 outlier
        scores = abod.decision_scores_

        processed = pd.DataFrame({
            'Anomaly': labels,
            'Anomaly_Score': scores,
            '_orig_index': df_numeric['_orig_index'].values
        })

        df_with_outliers = map_predictions_to_original(df_original, processed, anomaly_col='Anomaly', score_col='Anomaly_Score')

        n_out = int(df_with_outliers['Anomaly'].sum())
        print(f"PyOD ABOD selesai — ditemukan {n_out} outlier dari {len(df_with_outliers)} baris.")
        print("Contoh outlier (max 10):")
        print(df_with_outliers[df_with_outliers['Anomaly'] == 1].head(10))

        df_clean = df_with_outliers[df_with_outliers['Anomaly'] != 1].copy()
        df_clean = df_clean.drop(columns=['Anomaly', 'Anomaly_Score'])
        print(f"Jumlah baris setelah bersih: {len(df_clean)}")

    except Exception as e_pyod:
        print("\n❌ Fallback PyOD juga gagal. Error:")
        traceback.print_exc()
        raise RuntimeError("Kedua PyCaret & PyOD gagal — periksa instalasi paket dan versi scikit-learn.") from e_pyod

# ----------------- Opsional: simpan hasil -----------------
# df_with_outliers.to_csv("with_outliers.csv", index=False)
# df_clean.to_csv("clean_data.csv", index=False)

print("\n✅ Selesai. Data dengan label outlier ada di 'df_with_outliers'; dataset bersih ada di 'df_clean'.")



=== Mencoba PyCaret ABOD ===

⚠️ PyCaret gagal atau tidak tersedia. Error:

=== Menggunakan PyOD ABOD (fallback) ===
PyOD ABOD selesai — ditemukan 5 outlier dari 100 baris.
Contoh outlier (max 10):
    id         x1         x2 category  Anomaly  Anomaly_Score
0    0  20.000000   2.169259        A      1.0  -1.823226e-07
1    1  -0.138264 -30.000000        A      1.0  -9.315983e-08
6    6   1.579213   8.772372        A      1.0  -9.938134e-02
13  13  -1.913280   9.926484        A      1.0  -3.822176e-02
79  79  -1.987569  10.440338        B      1.0  -3.302782e-02
Jumlah baris setelah bersih: 95

✅ Selesai. Data dengan label outlier ada di 'df_with_outliers'; dataset bersih ada di 'df_clean'.


Traceback (most recent call last):
  File "C:\Users\user\AppData\Local\Temp\ipykernel_11576\4088490512.py", line 65, in <module>
    exp = setup(
          ^^^^^^
TypeError: setup() got an unexpected keyword argument 'silent'


In [7]:
# Pilih dataset bersih yang ingin Anda simpan (misalnya, hasil dari ABOD PyOD)
dataset_untuk_disimpan = df_bersih_abod
nama_file_baru = 'dataset_bersih_tanpa_outlier.csv'

try:
    # Simpan DataFrame ke file CSV
    # index=False agar nomor index dari DataFrame tidak ikut tersimpan sebagai kolom
    dataset_untuk_disimpan.to_csv(nama_file_baru, index=False)
    
    print(f"\n💾 Dataset baru berhasil disimpan sebagai '{nama_file_baru}'.")
    print("Anda sekarang dapat menggunakan file ini untuk analisis lebih lanjut.")

except Exception as e:
    print(f"❌ Gagal menyimpan file: {e}")


💾 Dataset baru berhasil disimpan sebagai 'dataset_bersih_tanpa_outlier.csv'.
Anda sekarang dapat menggunakan file ini untuk analisis lebih lanjut.
